In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
# 한글 쓰는 방법
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = "/System/Library/Fonts/AppleSDGothicNeo.ttc"
font_name = fm.FontProperties(fname=font_location).get_name()
print(font_name)
matplotlib.rc('font', family=font_name)
import os
# 현재 working directory 확인 방법
print(os.getcwd()) # 주피터에서
#pwd # 터미널에서 
# 데이터 있는 경로 확인
path = "/Users/jungtaekchoi/Google Drive/My Drive/fastcamp/datas"
# 경로 바꾸는 방법
os.chdir(path)
# 리스트 목록 확인
!ls

Apple SD Gothic Neo
/Volumes/GoogleDrive/My Drive/fastcamp/MGS_AI_4/dev/Fastcampus-EDA/cjt
Fastcampus_data_after_cleaning.csv
country.csv
sakila-data.sql
sakila-schema.sql
world.sql
수업.sql
수업2-2.sql
수업2.sql
데이터분석프로젝트_VER2_210329_210926.xlsx


/var/folders/k4/qhbh8yls6h349rdp_mfbt60w0000gn/T/ipykernel_78207/2293192428.py:9: MatplotlibDeprecationWarning: 
The get_fontconfig_fonts function was deprecated in Matplotlib 3.5 and will be removed two minor releases later.
  fm.get_fontconfig_fonts()


In [2]:
df = pd.read_csv(path + "/Fastcampus_data_after_cleaning.csv")
#df.dropna(inplace = True)

In [ ]:
df.loc[(df.transaction_amount != df.sale_price) & 
       (df.coupon_title == "없음") &
       (df.type == "PAYMENT") &
       (df.state == "COMPLETED"), "coupon_title"] = "차액" 
df["coupon_discount_amount"] = df.sale_price.sub(df.transaction_amount) 

df.drop(df.loc[df.category_title == "관리", :].index, inplace = True)
df.loc[(df.category_title == "교육") &
       (df.type == "PAYMENT") &
       (df.state == "COMPLETED"), :].coupon_discount_amount.unique()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plus_one(series) :
    return series + 1

def differencing_df(df, sub):
    target_series = df.loc[sub,].apply(plus_one)
    df = pd.DataFrame({"매출" : target_series})
    # 차분
    df['simple'] = df.diff(1)
    # 로그
    df['log'] = np.log(df["매출"])
    # 로그차분
    df['log_diff'] = np.log(df["매출"]).diff(1)
    # plot
    fig = make_subplots(4,1)
    fig.add_trace(go.Scatter(x=df.index, y=df.매출,name = "매출"),row=1,col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df.simple,name = "차분값"), row=2,col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df.log,name = "로그변환"), row=3,col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df.log_diff,name = "로그 차분"), row=4,col=1)

    fig.update_layout(height=600, width=800, title_text= f"{sub}")
    fig.show()
    

In [ ]:
for sub in sub_name : 
    differencing_df(ctg_df, sub = sub)

In [3]:
from matplotlib import rc
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def differencing_df2(df, sub):
    target_series = df.loc[sub,].apply(plus_one)
    df = pd.DataFrame({"매출" : target_series})
    # 차분
    df['simple'] = df.diff(1)
    # 로그
    df['log'] = np.log(df["매출"])
    # 로그차분
    df['log_diff'] = np.log(df["매출"]).diff(1)
    
    # plot
    rc('font', family='AppleGothic')
    plt.rcParams['axes.unicode_minus'] = False

    fig,ax = plt.subplots(4,2, figsize=(12,16))
    ax[0,0].plot(df['매출'])
    ax[0,0].set_title('매출')
    ax[1,0].plot(df['simple'])
    ax[1,0].set_title('차분')
    ax[2,0].plot(df['log'])
    ax[2,0].set_title('로그 변환')
    ax[3,0].plot(df['log_diff'])
    ax[3,0].set_title('로그 차분')

    plot_pacf(df['매출'],ax=ax[0,1])
    plot_pacf(df['simple'],ax=ax[1,1])
    plot_pacf(df['log'][1:],ax=ax[2,1])
    plot_pacf(df['log_diff'][1:],ax=ax[3,1])
    
    plt.show()
    

In [ ]:
for sub in sub_name : 
    differencing_df2(ctg_df, sub = sub)